In [39]:
%load_ext autoreload
%autoreload 2
# import re
from sqlitedict import SqliteDict
import argparse
import xgboost as xgb
import sys,os
import json

from scripts.utils import normalise_title
from scripts.utils import getPageDict,process_page

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
lang = 'de'
page_title = "Garnet Carter"
threshold = 0.9

In [45]:
# ## open datasets as shelve
anchors = SqliteDict("./data/{0}/{0}.anchors.sqlite".format(lang))
pageids = SqliteDict("./data/{0}/{0}.pageids.sqlite".format(lang)) 
redirects = SqliteDict("./data/{0}/{0}.redirects.sqlite".format(lang)) 
word2vec = SqliteDict("./data/{0}/{0}.w2vfiltered.sqlite".format(lang))

## load trained model
import xgboost as xgb
model = xgb.XGBClassifier()  # init model
model.load_model('./data/{0}/{0}.linkmodel.json'.format(lang))  # load data

In [46]:
# Get the wikitext of a page
page_title = page_title.replace(' ','_')
page_dict = getPageDict(page_title,lang)
wikitext = page_dict['wikitext']
pageid = page_dict['pageid']
revid = page_dict['revid']

In [47]:
## get the added links
added_links = process_page(wikitext, page_title, anchors, pageids, redirects, word2vec, model, threshold = threshold, return_wikitext = False)

In [48]:
added_links

[{'linkTarget': 'Fairyland',
  'anchor': 'Fairyland',
  'probability': 0.999219536781311,
  'startOffset': 359,
  'endOffset': 368},
 {'linkTarget': 'Vereinigte Staaten',
  'anchor': 'USA',
  'probability': 0.984917938709259,
  'startOffset': 655,
  'endOffset': 658}]

In [49]:
## we can also get the wikitext with the new links already inserted
wikitext_new = added_links = process_page(wikitext, page_title, anchors, pageids, redirects, word2vec, model, threshold = threshold, return_wikitext = True)
print(wikitext_new)

'''Garnet Carter''' (* [[9. Februar]] [[1883]] in [[Sweetwater (Tennessee)|Sweetwater]], [[Lewis County (Tennessee)|Lewis County]], [[Tennessee]]; † [[21. Juli]] [[1954]] in [[Lookout Mountain]], [[Hamilton County (Tennessee)|Hamilton County]], Tennessee) war Erfinder des [[Miniaturgolf]]. 

== Karriere ==
1927 eröffnete er seinen ersten Bahnengolfplatz im [[Fairyland|Fairyland|pr=0.99921954]] Club in  [[Lookout Mountain]]. Zu seiner Überraschung wurde der Platz von Erwachsenen übernommen, die Spaß an dem miniaturisierten Fairway hatten. 

Er begann darauf weitere Minigolfplätze zu produzieren unter dem patentierten Namen ''Tom Thumb Golf''. 1930 gab es 25.000 Plätze in den [[Vereinigte Staaten|USA|pr=0.98491794]]. 1932 verkaufte er die Rechte und eröffnete zusammen mit seiner Frau [[Rock City (Georgia)|Frieda's Rock City Garden]].

== Weblinks ==
{{Findagrave|7481668}}

{{SORTIERUNG:Carter, Garnet}}
[[Kategorie:Unternehmer (Vereinigte Staaten)]]
[[Kategorie:US-Amerikaner]]
[[Kategorie

In [50]:
anchors.close()
pageids.close()
redirects.close()
word2vec.close()